# Pythonopdracht - Programming for Data Science - Jelle van Barneveld

## Installatie-instructies

TODO de werking van "requirements.txt" controleren. Zodra hij werkt kan de eerste bullet worden weggehaald

Om het onderstaande script succesvol te kunnen runnen dien je de volgende stappen te ondernemen. Hierbij gaat men ervan uit dat het script geraadpleegd wordt door een Windows-apparaat.
- Open een Anaconda Prompt (zoek op via het startmenu) en typ daar het volgende: <code>pip install selenium</code>. Druk vervolgens op enter en wacht totdat je opnieuw iets kan invoeren. Sluit Anaconda Prompt vervolgens weer af.
- Open Google Chrome en klik op de drie puntjes bovenaan. Ga via "Help" naar "Over Google Chrome". Je ziet een versienummer, onthoud de cijfers t/m de eerste punt.
- Ga naar https://chromedriver.chromium.org/downloads en klik op het onthouden versienummer. Kies voor de Windowsversie, download de zipmap en pak deze uit. Verplaats de applicatie "chromedriver" naar C:\Program Files (x86)

Als het goed is runnen na het doorlopen van deze stappen de onderstaande 2 codeblokken succesvol. Zij vormen dus een goede test of de installatiestappen goed zijn uitgevoerd, en bevatten tegelijkertijd benodigdheden om de rest van de code te kunnen runnen.

In [23]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchWindowException

from bokeh.plotting import figure, show
from bokeh.models import LogColorMapper, ColumnDataSource
from bokeh.palettes import RdYlGn as correct_palette

import pandas as pd
import requests
import warnings
import geopandas


In [24]:
pad = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(pad)
warnings.simplefilter(action = 'ignore', category = FutureWarning)

C:\Users\chris\AppData\Local\Temp\ipykernel_10264\2948619622.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(pad)


## Fase 1: Business Understanding

### 1a: Aanleiding

Iedere zomer ga ik met ongeveer zeven a acht vrienden op vakantie in Nederland, we huren dan altijd een huisje. Op internet zoeken we een aantal maanden voor de schoolvakantie een huisje dat ons hiervoor geschikt lijkt. Elk van ons controleert deze geschiktheid volgens andere eisen. Grofweg kunnen deze in 2 categorieën worden opgedeeld:
- Eisen aan het huisje zelf: denk aan aantallen slaapkamers, WiFi-mogelijkheden, enzovoort. Deze informatie wordt aangeboden door de website natuurhuisje.nl.
- Eisen aan de omgeving van het huisje: dit omvat de aanwezigheid van een supermarkt en verschillende activiteiten, bijvoorbeeld pretparken, zwembaden, klimbossen, enzovoort. Deze informatie halen we voor het grootste gedeelte uit Google Maps.

Het zoekproces om een huisje te vinden dat aan alle eisen voldoet is tijdrovend: gemiddeld doen we er twee weken over om te beslissen wat het meest geschikte huisje is. Dit wordt met name veroorzaakt door het feit dat we niet weten wat binnen Nederland het meest aantrekkelijke zoekgebied is: om provinciale verschillen te achterhalen moeten we, zoals eerder vermeld, meerdere websites raadplegen. In dit beslisproces moeten we dus continu switchen tussen huisjes- en uitjesinformatie per provincie en daarbij zeer alert zijn op alle voorwaarden die we hebben gesteld.

### 1b: Het onderzoek: hoofd- en deelvragen

Dit script is geschreven om (de hiaten in de) bovengenoemde situatie te verbeteren door het beantwoorden van de volgende hoofdvraag: <b>"In welke provincie kan je het beste een huisje huren?"</b>. Hierdoor krijgen we snel inzicht in zowel de huisjes- als omgevingsverschillen per provincie, zodat we het "zoekgebied" kleiner kunnen maken ter verkleining van de zoektijd.

Om ervoor te zorgen dat het antwoord van deze vraag volledig volgens meerdere criteria beantwoord wordt, is deze opgesplitst in 4 deelvragen:
- In welke provincie staan de huisjes met de hoogste gemiddelde persoonscapaciteit?
- In welke provincie staan de huisjes met de laagste gemiddelde afstand tot een supermarkt?
- In welke provincie staan de huisjes met de laagste gemiddelde afstand tot een restaurant?
- In welke provincie staan de huisjes met de laagste gemiddelde afstand tot een attractie?

### 1c: Doelstelling

Met het geschreven script wil men het bovengenoemde zoek- en beslisproces om een huisje te vinden dat aan alle voorwaarden voldoet met de helft (een week) te verminderen. Doordat provinciale verschillen snel zichtbaar worden door een allesomvattende dataset, en er dus niet meer geschakeld hoeft te worden tussen verschillende websites, kunnen er naar verwachting sneller huisjes gevonden en gekeurd worden.

### 1d: Fasering & aanpak

Voor het grootste gedeelte van deze opdracht zijn de stappen van het veelgebruikte CRISP-DM-procesmodel (Cross-industry Stand Process for Data Mining): op een "watervalwijze" doorlopen. Dit houdt in dat per projectfase een CRISP-DM-fase werd uitgevoerd, zonder tussentijds terug te gaan naar een eerdere CRISP-DM-fase.
Er zijn twee redenen waarom hierbij voor een waterval- boven een iteratieve aanpak is gekozen (verantwoord volgens de Ralph Staceymatrix):
- De tijdens "Business Understanding" geïnventariseerde eisen bleven gedurende dit hele traject ongewijzigd.
- De gebruikte technieken waren van tevoren al bekend (namelijk Python-only) en ook zij zijn niet veranderd in de loop der tijd. Weliswaar moest men een aantal onbekende Python-packages gebruiken (selenium, bokeh, enzovoort), maar hier was voldoende informatie over te vinden op het internet.

Hieronder volgt per CRISP-DM-stap een globale beschrijving van de toepassingswijze binnen de projectfasen:
- <b>Fase 1: Business Understanding:</b> er is een aanleiding, opdracht en doelstelling geformuleerd, waarmee men de toegevoegde waarde van dit project rechtvaardigt.
- <b>Fase 2: Data Understanding:</b> in deze tweede fase is er data gelokaliseerd d.m.v. het verzamelen van handige databronnen (een website, een API & een gedownload geojson-bestand) die elk veel informatie bevatten. Wat de website (natuurhuisje.nl) betreft zijn d.m.v. de "Inspecteren"-optie van Google Chrome de relevante achterliggende HTML-structuren geanalyseerd, om vast te stellen naar welke elementen Python moet zoeken voor het verkrijgen van de juiste informatie.
- <b>Fase 3: Data Preparation:</b> hier wordt de relevante geïnventariseerde data daadwerkelijk opgehaald en in een dataset gezet. Het bewaken van de volledigheid en integriteit van de data staat hierbij centraal: continu wordt de uiteindelijke dataset in Visual Studio Code vergeleken met bijvoorbeeld webpagina's in Chrome om vast te stellen of álle en de júiste data goed overgekomen zijn.
- <b>Fase 4: Evaluation & Deployment:</b> hier worden in een interactieve kaart visuele analysetechnieken gecombineerd om de betreffende deelvragen, en daarmee de hoofdvraag, te beantwoorden. Hierbij wordt zoveel mogelijk geprobeerd om de gestelde hypotheses te ontkrachten: je mag er immers pas vanuit gaan dat iets waar is, als je het tegendeel niet meer kan bewijzen. De gemaakte kaart wordt ten slotte gepubliceerd in een HTML-bestand dat de gebruiker naar eigen inzicht kan raadplegen.

De CRISP-DM-fase "Modelling" is in dit project buiten beschouwing gelaten, omdat er geen Machine-Learningtechnieken zijn toegepast.

## Fase 2: Data Understanding

### 2a: De natuurhuisjes op zich

Voor het verzamelen van data over de natuurhuisjes van de website natuurhuisje.nl wordt de functie verzamel_huiselementen(...) aangeroepen. Deze doorloopt de resultatenlijst van de URL https://www.natuurhuisje.nl/vakantiehuizen/nederland , die verspreid is over meerdere pagina's. Per pagina haalt deze functie een grote hoeveelheid HTML-code op, hiermee worden vervolgens 2 aanvullende functies aangeroepen: 
- haal_woonplaatsen_uit_resultatenlijst(...): de woonplaats van elk huisje wordt geëxtraheerd vanuit het betreffende HTML-element in de resultatenlijst en in een Pythonlijst gezet. Deze lijst wordt geretourneerd door de functie.
- maak_detailpagina_frame(): naar elk huisje wordt "doorgeklikt" om op diens pagina te komen met detailinformatie. Deze wordt gebundeld in een dataframe, welke ten slotte wordt teruggegeven.

De drie bovengenoemde functies zijn hieronder in omgekeerde volgorde gedefinieerd, omdat verzamel_huiselementen(...) geen functies kan aanroepen die nog niet eerder zijn gemaakt. Onder elke functie staat een stapsgewijze beschrijving van de werking ervan, met een gegeven URL als voorbeeld.

In [25]:
# def maak_detailpagina_frame(link):
#     url = requests.get(link)
#     df = pd.concat([pd.read_html(url.text)[0], pd.read_html(url.text)[1]], axis = 0)
    
#     df = df.transpose()
#     df.columns = df.iloc[0]
#     df = df.iloc[1:]
    
#     return df

In [26]:
def extraheer_detailinfo(huidige_link, eigenschappenlijst):
    url = requests.get(huidige_link)
    print(list(pd.read_html(url.text)[0]))
    
    # df = pd.concat([pd.read_html(url.text)[0], pd.read_html(url.text)[1]], axis = 0)
    
    # df = df.transpose()
    # df.columns = df.iloc[0]
    # df = df.iloc[1:]
    
    # return df

Stel dat de parameter "link" de volgende URL als waarde heeft: https://www.natuurhuisje.nl/vakantiehuisje/60401 . ALs je op deze URL klikt zie je dat er 2 tabellen zijn die allebei relevante eigenschappen bevatten om in de uiteindelijke dataset op te nemen. Beide tabellen worden ingelezen en in eerste instantie gebeurt dat "row-wise": elke huiseigenschap krijgt zijn eigen rij (met in de eerste kolom de betreffende eigenschap en in de tweede kolom diens bijbehorende waarde). Met de onderste coderegels worden deze eigenschappen naar de kolommen verplaatst, waardoor je in het dataframe slechts 1 regel overhoudt, en wordt het dataframe geretourneerd.

In [27]:
def haal_woonplaatsen_uit_resultatenlijst(html_brij):
    alle_woonplaatsen = []

    for html_element in html_brij:
        opgesplitste_html_elementen = html_element.text.split("\n")
        woonplaats_list = [s for s in opgesplitste_html_elementen if "Natuurhuisje in" in s]
        woonplaats_str = woonplaats_list[0]
        alle_woonplaatsen.append(woonplaats_str)
    
    return alle_woonplaatsen

Aan deze functie wordt allereerst de parameter "html_brij" weergegeven: dit is HTML-code die verantwoordelijk is voor het correct weergeven van de huisjes op een bepaalde pagina met zoekresultaten (zie https://www.natuurhuisje.nl/vakantiehuizen/nederland?skip=0 voor een voorbeeld). Op het moment dat deze functie wordt aangeroepen is de "brij" al gefilterd: alleen de HTML-code die betrekking heeft op de huisjes zit hierin; alle andere code (head, footer, meta, enzovoort) wordt buiten beschouwing gelaten.

In de for-loop wordt elk huisje doorlopen: diens elementen worden per huisje eerst in een lijst gezet. Vervolgens wordt het element gekozen dat de substring "Natuurhuisje in" bevat, omdat daar altijd de woonplaats in staat. Al deze woonplaatsen worden ten slotte toegevoegd aan een allesomvattende lijst, die wordt teruggegeven. Deze functie wordt dus aangeroepen per zoekresultaatpagina, zodat er per pagina dus een lijst ontstaat met correcte betreffende woonplaatsen.

In [28]:
class Huisje:

    def __init__(self, natuurhuisje_id, woonplaats, regio, land, aantal_personen, slaapkamers, type, huursituatie, check_in, check_out):
        self.natuurhuisje_id = natuurhuisje_id
        self.woonplaats = woonplaats
        self.regio = regio
        self.land = land
        self.aantal_personen = aantal_personen
        self.slaapkamers = slaapkamers
        self.type = type
        self.huursituatie = huursituatie
        self.check_in = check_in
        self.check_out = check_out

In [29]:
# def verzamel_huiselementen(driver, zoekmethode, zoekterm):
#     html_brij = driver.find_elements(zoekmethode, zoekterm)
#     alle_woonplaatsen = haal_woonplaatsen_uit_resultatenlijst(html_brij)
#     df_lijst = []
    
#     for i, huidige_woonplaats in enumerate(alle_woonplaatsen):
#         huidig_webelement = html_brij[i] 
        
#         homepage_df = pd.DataFrame(columns = ['Natuurhuisje-ID', 'Woonplaats'])
#         huidig_id = huidig_webelement.get_attribute('data-house-id')
#         homepage_df.loc[len(homepage_df)] = [huidig_id, huidige_woonplaats]

#         huidige_link = 'https://www.natuurhuisje.nl/vakantiehuisje/' + str(huidig_id)
#         detailpagina_df = maak_detailpagina_frame(huidige_link)

#         home_en_detailpagina_df = pd.merge(homepage_df, detailpagina_df, on = 'Natuurhuisje-ID', how = 'inner')
#         df_lijst.append(home_en_detailpagina_df)
        
#     return df_lijst

In [30]:
def maak_huisjes(driver, zoekmethode, zoekterm):
    html_brij = driver.find_elements(zoekmethode, zoekterm)
    alle_woonplaatsen = haal_woonplaatsen_uit_resultatenlijst(html_brij)
    huisjeslijst = []
    
    for i, huidige_woonplaats in enumerate(alle_woonplaatsen):
        huidig_webelement = html_brij[i]
        huidig_id = huidig_webelement.get_attribute('data-house-id')
        eigenschappenlijst = [huidig_id]
        
        huidige_link = 'https://www.natuurhuisje.nl/vakantiehuisje/' + str(huidig_id)
        detailpagina_df = extraheer_detailinfo(huidige_link, eigenschappenlijst)

        home_en_detailpagina_df = pd.merge(homepage_df, detailpagina_df, on = 'Natuurhuisje-ID', how = 'inner')
        df_lijst.append(home_en_detailpagina_df)
    

Ook deze functie wordt, net als de vorige, aangeroepen per pagina met zoekresultaten op de website natuurhuisje.nl (referentie: https://www.natuurhuisje.nl/vakantiehuizen/nederland?skip=0). Eerst wordt de html_brij gedefinieerd en ook meteen gefilterd op zichtbare huisjes met de ingegeven zoekmethode & zoekterm. Ook worden eerst alle woonplaatsen op deze webpagina opgehaald met haal_woonplaatsen_uit_resultatenlijst(...) en in een lijst gezet.

Vervolgens wordt elk zoekresultaat doorlopen: per resultaat wordt er eerst een leeg dataframe gemaakt die vervolgens wordt gevuld met gegevens vanuit de zoekresultatenlijst: het juiste ID (afgelezen vanuit de HTML-code) en de woonplaats (afgelezen vanuit de bovengenoemde lijst). Hierna wordt de juiste link gegenereerd, waarmee maak_detailpagina_frame(...) mee aangeroepen wordt om een tweede dataframe met detailinformatie te creëren. Deze wordt samengevoegd met het eerste dataframe: het resultaat van deze "merge" wordt aan een allesomvattende lijst van dataframes toegevoegd. Ten slotte wordt uit deze lijst een allesomvattend dataframe gemaakt, die door deze functie wordt teruggegeven.

In [31]:
while(True):
    
    try:
        aantal_resultaten_per_pagina = 20 #Deze staat vast op natuurhuisje.nl
        geschat_aantal_minuten_per_pagina = 0.25
        aantal_paginas = int(input("Hoeveel pagina's wil je inlezen? Elke pagina bevat " + str(aantal_resultaten_per_pagina) + " huisjes en duurt ongeveer " + str(geschat_aantal_minuten_per_pagina) + " minuten om in te lezen"))
        
        totaal_aantal_huisjes = aantal_paginas * aantal_resultaten_per_pagina
        geschatte_tijdsduur_totaal = aantal_paginas * geschat_aantal_minuten_per_pagina

        print("Er zullen " + str(aantal_paginas) + " pagina's ingelezen worden, wat resulteert in " + str(totaal_aantal_huisjes) + " in de dataset. Dit zal ongeveer " + str(geschatte_tijdsduur_totaal) + " minuten kosten.")
        print("Wil je het laadproces tussentijds afbreken? Sluit de Chrome Webviewer, die zichzelf opent aan het begin van het laadproces, dan af.")
        
        break
    except ValueError:
        print("Voer een geheel getal in!")

Er zullen 1 pagina's ingelezen worden, wat resulteert in 20 in de dataset. Dit zal ongeveer 0.25 minuten kosten.
Wil je het laadproces tussentijds afbreken? Sluit de Chrome Webviewer, die zichzelf opent aan het begin van het laadproces, dan af.


In [32]:
aantal_verwerkte_huisjes = 0
df_lijst = []

In [33]:
while(aantal_verwerkte_huisjes < totaal_aantal_huisjes):
    
    try:
        url = "https://www.natuurhuisje.nl/vakantiehuizen/nederland?skip=" + str(aantal_verwerkte_huisjes)
        driver.get(url)
    
        alle_zoekresultaten = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'search__result-list')))   
        alle_huisjes_op_een_pagina = maak_huisjes(driver, 'xpath', "//article[@class ='card card--horizontal card--house-search']")
        df_lijst += alle_huisjes_op_een_pagina
        
        aantal_verwerkte_huisjes += 20
        print(str(aantal_verwerkte_huisjes / 20) + " pagina's verwerkt")

    except NoSuchWindowException:
        break
    
df_lijst

[0, 1]


NameError: name 'homepage_df' is not defined

In [ ]:
huisjes = pd.concat(df_lijst, ignore_index = True)
huisjes

,Natuurhuisje-ID,Woonplaats,Regio,Land,Aantal personen,Slaapkamers,Type,Huursituatie,Ligging,Check-in,Check-out
0,71433,Natuurhuisje in Erp,Noord-Brabant,Nederland,4,1,Glamping,Vrijstaand,Alleenstaand,15:00 - 22:00,07:00 - 12:00
1,71464,Natuurhuisje in Stegeren,Overijssel,Nederland,5,3,Bungalow,Vrijstaand,Kleinschalig vakantiepark,15:00 - 23:00,07:00 - 11:00
2,71446,Natuurhuisje in Terkaple,Friesland,Nederland,4,2,Vakantiehuis,Vrijstaand,Op een erf,15:00 - 21:00,08:00 - 11:00
3,71499,Natuurhuisje in Putten,Gelderland,Nederland,6,2,Glamping,Vrijstaand,Op een landgoed,16:00 - 21:00,08:00 - 11:00
4,71465,Natuurhuisje in Tjerkwerd,Friesland,Nederland,4,3,Vakantiehuis,Vrijstaand,Op een erf,15:00 - 22:00,07:00 - 11:00
5,71483,Natuurhuisje in IJhorst,Drenthe,Nederland,6,3,Vakantiehuis,Vrijstaand,Kleinschalig vakantiepark,15:00 - 20:00,07:00 - 11:00
6,71467,Natuurhuisje in Mariënberg,Overijssel,Nederland,6,4,Vakantiehuis,Vrijstaand,In een dorpje,15:00 - 20:00,07:00 - 10:00
7,71523,Natuurhuisje in Hommerts,Friesland,Nederland,4,2,Safaritent,Vrijstaand,Kleinschalig vakantiepark,15:00 - 22:00,07:00 - 10:00
8,71553,"Natuurhuisje in Zeeland, Maashorst",Noord-Brabant,Nederland,2,1,Pipowagen,Vrijstaand,Alleenstaand,15:00 - 22:00,07:00 - 11:00
9,60335,Natuurhuisje in Egmond aan den Hoef,Noord-Holland,Nederland,2,1,Yurt,Vrijstaand,Alleenstaand,15:00 - 19:00,09:00 - 11:00


In [ ]:
Stop hier

SyntaxError: invalid syntax (384373577.py, line 1)

### 2b: Supermarkten en uitjes

In [ ]:
def get_odata(target_url):
    data = pd.DataFrame()
    while target_url:
        r = requests.get(target_url).json()
        data = data.append(pd.DataFrame(r['value']))
        
        if '@odata.nextLink' in r:
            target_url = r['@odata.nextLink']
        else:
            target_url = None
            
    return data

In [ ]:
voorzieningen = pd.DataFrame()
huidig_jaar = datetime.date.today().year

while(voorzieningen.shape[0] == 0):
    huidig_jaar_str = str(huidig_jaar)

    voorzieningen = get_odata("https://opendata.cbs.nl/ODataApi/odata/80305ned/TypedDataSet")
    voorzieningen = voorzieningen.loc[voorzieningen['Perioden'].str[0:4] == huidig_jaar_str, :]
    huidig_jaar -= 1

voorzieningen

,ID,RegioS,Perioden,AfstandTotHuisartsenpraktijk_1,Binnen1Km_2,Binnen3Km_3,Binnen5Km_4,AfstandTotHuisartsenpost_5,AfstandTotApotheek_6,AfstandTotZiekenhuis_7,...,Binnen5Km_116,Binnen10Km_117,Binnen20Km_118,AfstandTotSauna_119,AfstandTotZonnebank_120,AfstandTotAttractie_121,Binnen10Km_122,Binnen20Km_123,Binnen50Km_124,AfstandTotBrandweerkazerne_125
16,16,NL01,2022JJ00,1.1,1.3,7.8,16.9,NaN,1.3,4.9,...,1.3,2.9,7.2,7.9,4.1,5.3,3.3,9.6,43.9,2.2
33,33,LD01,2022JJ00,1.5,0.9,4.2,7.6,NaN,1.8,8.9,...,0.7,1.0,2.1,12.2,8.9,7.2,1.3,4.0,21.0,2.8
50,50,LD02,2022JJ00,1.2,0.9,4.5,8.5,NaN,1.4,5.3,...,0.6,1.3,3.6,10.3,4.4,6.1,2.0,5.4,33.5,2.3
67,67,LD03,2022JJ00,0.9,1.6,11.1,25.4,NaN,1.1,3.8,...,1.8,4.3,10.9,5.9,3.1,4.6,4.5,13.5,57.6,1.9
84,84,LD04,2022JJ00,1.1,1.0,5.2,10.7,NaN,1.4,5.1,...,0.9,2.0,4.8,8.0,4.0,5.3,2.7,7.7,33.9,2.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9383,9383,GM0879,2022JJ00,2.3,0.3,0.7,0.9,NaN,2.3,11.0,...,0.0,0.3,4.0,15.7,3.6,6.6,0.9,5.6,23.5,2.5
9400,9400,GM0301,2022JJ00,0.8,1.1,7.0,11.3,NaN,1.1,2.9,...,1.9,2.0,3.8,3.9,2.2,3.0,1.0,2.0,34.5,2.0
9417,9417,GM1896,2022JJ00,1.0,0.9,1.8,2.9,NaN,1.2,13.5,...,0.0,0.0,3.1,21.2,11.2,13.3,0.0,5.3,34.7,1.3
9434,9434,GM0642,2022JJ00,0.7,1.3,6.3,16.8,NaN,0.9,2.6,...,0.3,2.8,9.9,5.3,1.6,3.7,1.2,10.2,66.7,1.7


In [ ]:
woonplaatsen = get_odata("https://opendata.cbs.nl/ODataApi/odata/80305ned/RegioS")
woonplaatsen = woonplaatsen.loc[woonplaatsen['Key'].str[0:2] == 'GM', ['Key', 'Title']]
woonplaatsen

,Key,Title
57,GM1680,Aa en Hunze
58,GM0738,Aalburg
59,GM0358,Aalsmeer
60,GM0197,Aalten
61,GM0480,Ter Aar
...,...,...
551,GM0879,Zundert
552,GM0301,Zutphen
553,GM1896,Zwartewaterland
554,GM0642,Zwijndrecht


In [ ]:
voorzieningen_en_woonplaatsen = pd.merge(voorzieningen, woonplaatsen, left_on = 'RegioS', right_on = 'Key', how = 'inner')
voorzieningen_en_woonplaatsen

,ID,RegioS,Perioden,AfstandTotHuisartsenpraktijk_1,Binnen1Km_2,Binnen3Km_3,Binnen5Km_4,AfstandTotHuisartsenpost_5,AfstandTotApotheek_6,AfstandTotZiekenhuis_7,...,Binnen20Km_118,AfstandTotSauna_119,AfstandTotZonnebank_120,AfstandTotAttractie_121,Binnen10Km_122,Binnen20Km_123,Binnen50Km_124,AfstandTotBrandweerkazerne_125,Key,Title
0,985,GM1680,2022JJ00,2.4,0.4,0.9,1.6,NaN,2.5,10.8,...,1.8,13.5,10.9,7.1,1.2,5.3,24.8,2.7,GM1680,Aa en Hunze
1,1002,GM0738,2022JJ00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GM0738,Aalburg
2,1019,GM0358,2022JJ00,0.9,1.0,3.3,8.9,NaN,1.0,5.2,...,12.2,4.7,4.0,2.9,3.4,16.2,78.8,2.9,GM0358,Aalsmeer
3,1036,GM0197,2022JJ00,1.5,1.1,2.6,3.2,NaN,1.5,13.7,...,1.2,13.9,4.4,14.2,0.1,1.5,12.7,1.7,GM0197,Aalten
4,1053,GM0480,2022JJ00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GM0480,Ter Aar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,9383,GM0879,2022JJ00,2.3,0.3,0.7,0.9,NaN,2.3,11.0,...,4.0,15.7,3.6,6.6,0.9,5.6,23.5,2.5,GM0879,Zundert
495,9400,GM0301,2022JJ00,0.8,1.1,7.0,11.3,NaN,1.1,2.9,...,3.8,3.9,2.2,3.0,1.0,2.0,34.5,2.0,GM0301,Zutphen
496,9417,GM1896,2022JJ00,1.0,0.9,1.8,2.9,NaN,1.2,13.5,...,3.1,21.2,11.2,13.3,0.0,5.3,34.7,1.3,GM1896,Zwartewaterland
497,9434,GM0642,2022JJ00,0.7,1.3,6.3,16.8,NaN,0.9,2.6,...,9.9,5.3,1.6,3.7,1.2,10.2,66.7,1.7,GM0642,Zwijndrecht


### 2c: Provincies

In [ ]:
provincies = geopandas.read_file("provinces.geojson")
provincies

,name,regioFacetId,level,geometry
0,Drenthe,tcm:106-353397-1024,3,"POLYGON ((6.41328 52.98552, 6.36252 53.03397, ..."
1,Flevoland,tcm:106-353410-1024,3,"POLYGON ((5.36115 52.67573, 5.37726 52.76481, ..."
2,Friesland (Fryslân),tcm:106-353417-1024,3,"POLYGON ((5.08707 53.32307, 5.10178 53.36803, ..."
3,Gelderland,tcm:106-353445-1024,3,"POLYGON ((5.60602 51.99416, 5.59200 52.00139, ..."
4,Groningen,tcm:106-353502-1024,3,"POLYGON ((6.28698 53.34138, 6.27368 53.34527, ..."
5,Limburg,tcm:106-353526-1024,3,"POLYGON ((5.91947 51.71767, 5.89957 51.72019, ..."
6,Noord-Brabant,tcm:106-353560-1024,3,"POLYGON ((5.20649 51.74148, 5.21635 51.74345, ..."
7,Noord-Holland,tcm:106-353628-1024,3,"POLYGON ((4.58332 52.53389, 4.59659 52.58401, ..."
8,Overijssel,tcm:106-353682-1024,3,"POLYGON ((6.10958 52.44053, 6.10229 52.44577, ..."
9,Utrecht,tcm:106-353708-1024,3,"POLYGON ((4.89218 52.16180, 4.85558 52.17892, ..."


## Fase 3: Data Preparation

In [ ]:
# huisjes = pd.read_excel("Huisjes op 13-06-2023.xlsx")
# huisjes = huisjes.loc[huisjes['Regio'] != 'Waddeneilanden', :]

# voorzieningen_en_woonplaatsen = pd.read_excel("Voorzieningen_en_woonplaatsen 13-06-2023.xlsx")
# voorzieningen_en_woonplaatsen

# provincies = geopandas.read_file("provinces.geojson")
# huisjes

### 3a: Huisjes schoonmaken

In [ ]:
huisjes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Natuurhuisje-ID  200 non-null    object
 1   Woonplaats       200 non-null    object
 2   Regio            200 non-null    object
 3   Land             200 non-null    object
 4   Aantal personen  200 non-null    object
 5   Slaapkamers      187 non-null    object
 6   Type             200 non-null    object
 7   Huursituatie     200 non-null    object
 8   Ligging          200 non-null    object
 9   Check-in         200 non-null    object
 10  Check-out        200 non-null    object
 11  Departement      4 non-null      object
dtypes: object(12)
memory usage: 18.9+ KB


In [ ]:
#TODO: wat doe ik met departement? Even kijken met kaart-tool!

In [ ]:
# huisjes.loc[huisjes['Departement'].notna(), :]

In [ ]:
huisjes['Woonplaats'] = huisjes['Woonplaats'].str.split(' in ', expand = True)[1]
huisjes

,Natuurhuisje-ID,Woonplaats,Regio,Land,Aantal personen,Slaapkamers,Type,Huursituatie,Ligging,Check-in,Check-out,Departement
0,71433,Erp,Noord-Brabant,Nederland,4,1,Glamping,Vrijstaand,Alleenstaand,15:00 - 22:00,07:00 - 12:00,NaN
1,71464,Stegeren,Overijssel,Nederland,5,3,Bungalow,Vrijstaand,Kleinschalig vakantiepark,15:00 - 23:00,07:00 - 11:00,NaN
2,71446,Terkaple,Friesland,Nederland,4,2,Vakantiehuis,Vrijstaand,Op een erf,15:00 - 21:00,08:00 - 11:00,NaN
3,71499,Putten,Gelderland,Nederland,6,2,Glamping,Vrijstaand,Op een landgoed,16:00 - 21:00,08:00 - 11:00,NaN
4,71465,Tjerkwerd,Friesland,Nederland,4,3,Vakantiehuis,Vrijstaand,Op een erf,15:00 - 22:00,07:00 - 11:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
195,68507,Valkenswaard,Noord-Brabant,Nederland,4,3,Blokhut,Vrijstaand,Alleenstaand,15:00 - 20:00,07:00 - 10:00,NaN
196,70118,Panningen,Limburg,Nederland,4,2,Bungalow,Vrijstaand,Alleenstaand,14:00 - 22:00,07:00 - 10:00,NaN
197,62241,Siegerswoude,Friesland,Nederland,6,1,Glamping,Vrijstaand,Kleinschalig vakantiepark,15:00 - 20:00,07:00 - 11:00,NaN
198,68116,Beerze,Overijssel,Nederland,6,2,Boerderij,Vrijstaand,Op een erf,15:00 - 22:00,07:00 - 11:00,NaN


### 3b: Voorzieningen en woonplaatsen schoonmaken

In [ ]:
relevante_attributen = ['Supermarkt', 'Treinstation', 'Zwembad', 'IJsbaan', 'Bioscoop', 'Sauna', 'Attractie']
relevante_kolommen = list(voorzieningen_en_woonplaatsen.columns[-2:])

for relevant_attribuut in relevante_attributen:

    for kolom in voorzieningen_en_woonplaatsen.columns:

        if(relevant_attribuut in kolom):
            relevante_kolommen.append(kolom)

relevante_kolommen

['Key',
 'Title',
 'AfstandTotGroteSupermarkt_20',
 'AfstandTotTreinstationsTotaal_101',
 'AfstandTotZwembad_104',
 'AfstandTotBioscoop_115',
 'AfstandTotSauna_119',
 'AfstandTotAttractie_121']

In [ ]:
voorzieningen_en_woonplaatsen = voorzieningen_en_woonplaatsen.loc[:, relevante_kolommen]
voorzieningen_en_woonplaatsen

,Key,Title,AfstandTotGroteSupermarkt_20,AfstandTotTreinstationsTotaal_101,AfstandTotZwembad_104,AfstandTotBioscoop_115,AfstandTotSauna_119,AfstandTotAttractie_121
0,GM1680,Aa en Hunze,2.2,12.4,9.8,11.8,13.5,7.1
1,GM0738,Aalburg,NaN,NaN,NaN,NaN,NaN,NaN
2,GM0358,Aalsmeer,0.8,8.2,3.0,8.7,4.7,2.9
3,GM0197,Aalten,1.4,4.4,5.3,14.5,13.9,14.2
4,GM0480,Ter Aar,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
494,GM0879,Zundert,1.2,13.0,2.5,12.7,15.7,6.6
495,GM0301,Zutphen,0.8,2.7,2.5,2.3,3.9,3.0
496,GM1896,Zwartewaterland,1.1,11.4,3.1,12.2,21.2,13.3
497,GM0642,Zwijndrecht,0.7,2.0,2.3,5.8,5.3,3.7


In [ ]:
voorzieningen_en_woonplaatsen = voorzieningen_en_woonplaatsen.rename(columns = {'Key' : 'Woonplaatscode', 'Title' : 'Woonplaats'})
voorzieningen_en_woonplaatsen

,Woonplaatscode,Woonplaats,AfstandTotGroteSupermarkt_20,AfstandTotTreinstationsTotaal_101,AfstandTotZwembad_104,AfstandTotBioscoop_115,AfstandTotSauna_119,AfstandTotAttractie_121
0,GM1680,Aa en Hunze,2.2,12.4,9.8,11.8,13.5,7.1
1,GM0738,Aalburg,NaN,NaN,NaN,NaN,NaN,NaN
2,GM0358,Aalsmeer,0.8,8.2,3.0,8.7,4.7,2.9
3,GM0197,Aalten,1.4,4.4,5.3,14.5,13.9,14.2
4,GM0480,Ter Aar,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
494,GM0879,Zundert,1.2,13.0,2.5,12.7,15.7,6.6
495,GM0301,Zutphen,0.8,2.7,2.5,2.3,3.9,3.0
496,GM1896,Zwartewaterland,1.1,11.4,3.1,12.2,21.2,13.3
497,GM0642,Zwijndrecht,0.7,2.0,2.3,5.8,5.3,3.7


In [ ]:
voorzieningen_en_woonplaatsen.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 499 entries, 0 to 498
Data columns (total 8 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Woonplaatscode                     499 non-null    object 
 1   Woonplaats                         499 non-null    object 
 2   AfstandTotGroteSupermarkt_20       345 non-null    float64
 3   AfstandTotTreinstationsTotaal_101  345 non-null    float64
 4   AfstandTotZwembad_104              345 non-null    float64
 5   AfstandTotBioscoop_115             345 non-null    float64
 6   AfstandTotSauna_119                345 non-null    float64
 7   AfstandTotAttractie_121            345 non-null    float64
dtypes: float64(6), object(2)
memory usage: 35.1+ KB


### 3c: Provincies schoonmaken

In [ ]:
lat_list_all_provinces = []
lon_list_all_provinces = []

for index, row in provincies.iterrows():
     lat_list_per_province = []
     lon_list_per_province = []
     
     for pt in list(row['geometry'].exterior.coords):
          lat_list_per_province.append(pt[0])
          lon_list_per_province.append(pt[1])
     
     lat_list_all_provinces.append(lat_list_per_province)
     lon_list_all_provinces.append(lon_list_per_province)

provincies['x'] = lat_list_all_provinces
provincies['y'] = lon_list_all_provinces
provincies

,name,regioFacetId,level,geometry,x,y
0,Drenthe,tcm:106-353397-1024,3,"POLYGON ((6.41328 52.98552, 6.36252 53.03397, ...","[6.413279838107642, 6.36252137268322, 6.367810...","[52.98552292584422, 53.033969027608755, 53.067..."
1,Flevoland,tcm:106-353410-1024,3,"POLYGON ((5.36115 52.67573, 5.37726 52.76481, ...","[5.361146853111475, 5.377259769555099, 5.63135...","[52.67572642054404, 52.76480520148312, 52.8030..."
2,Friesland (Fryslân),tcm:106-353417-1024,3,"POLYGON ((5.08707 53.32307, 5.10178 53.36803, ...","[5.087068926159327, 5.101778301669335, 5.16177...","[53.3230724825529, 53.36803427684854, 53.38613..."
3,Gelderland,tcm:106-353445-1024,3,"POLYGON ((5.60602 51.99416, 5.59200 52.00139, ...","[5.606017014104991, 5.591997672459713, 5.59111...","[51.99416434974128, 52.00139228996948, 52.0073..."
4,Groningen,tcm:106-353502-1024,3,"POLYGON ((6.28698 53.34138, 6.27368 53.34527, ...","[6.286980744809261, 6.273680380337391, 6.25404...","[53.34138175175612, 53.345271499820676, 53.348..."
5,Limburg,tcm:106-353526-1024,3,"POLYGON ((5.91947 51.71767, 5.89957 51.72019, ...","[5.919469547636648, 5.899566718712435, 5.88663...","[51.71767149749075, 51.720189936851206, 51.726..."
6,Noord-Brabant,tcm:106-353560-1024,3,"POLYGON ((5.20649 51.74148, 5.21635 51.74345, ...","[5.206494174004241, 5.216347456767744, 5.22503...","[51.74147718300763, 51.74345117135748, 51.7429..."
7,Noord-Holland,tcm:106-353628-1024,3,"POLYGON ((4.58332 52.53389, 4.59659 52.58401, ...","[4.583320874314651, 4.596585061787816, 4.61146...","[52.53389206786423, 52.58400753386461, 52.6555..."
8,Overijssel,tcm:106-353682-1024,3,"POLYGON ((6.10958 52.44053, 6.10229 52.44577, ...","[6.109580017263029, 6.102294636279321, 6.10335...","[52.440530726348904, 52.44577189750357, 52.456..."
9,Utrecht,tcm:106-353708-1024,3,"POLYGON ((4.89218 52.16180, 4.85558 52.17892, ...","[4.892178256568383, 4.855581829109592, 4.84392...","[52.16179791879903, 52.178917173752055, 52.180..."


In [ ]:
provincies = provincies.drop(['regioFacetId', 'level', 'geometry'], axis = 1)
provincies

,name,x,y
0,Drenthe,"[6.413279838107642, 6.36252137268322, 6.367810...","[52.98552292584422, 53.033969027608755, 53.067..."
1,Flevoland,"[5.361146853111475, 5.377259769555099, 5.63135...","[52.67572642054404, 52.76480520148312, 52.8030..."
2,Friesland (Fryslân),"[5.087068926159327, 5.101778301669335, 5.16177...","[53.3230724825529, 53.36803427684854, 53.38613..."
3,Gelderland,"[5.606017014104991, 5.591997672459713, 5.59111...","[51.99416434974128, 52.00139228996948, 52.0073..."
4,Groningen,"[6.286980744809261, 6.273680380337391, 6.25404...","[53.34138175175612, 53.345271499820676, 53.348..."
5,Limburg,"[5.919469547636648, 5.899566718712435, 5.88663...","[51.71767149749075, 51.720189936851206, 51.726..."
6,Noord-Brabant,"[5.206494174004241, 5.216347456767744, 5.22503...","[51.74147718300763, 51.74345117135748, 51.7429..."
7,Noord-Holland,"[4.583320874314651, 4.596585061787816, 4.61146...","[52.53389206786423, 52.58400753386461, 52.6555..."
8,Overijssel,"[6.109580017263029, 6.102294636279321, 6.10335...","[52.440530726348904, 52.44577189750357, 52.456..."
9,Utrecht,"[4.892178256568383, 4.855581829109592, 4.84392...","[52.16179791879903, 52.178917173752055, 52.180..."


In [ ]:
provincies = provincies.replace('Friesland (Fryslân)', 'Friesland')
provincies

,name,x,y
0,Drenthe,"[6.413279838107642, 6.36252137268322, 6.367810...","[52.98552292584422, 53.033969027608755, 53.067..."
1,Flevoland,"[5.361146853111475, 5.377259769555099, 5.63135...","[52.67572642054404, 52.76480520148312, 52.8030..."
2,Friesland,"[5.087068926159327, 5.101778301669335, 5.16177...","[53.3230724825529, 53.36803427684854, 53.38613..."
3,Gelderland,"[5.606017014104991, 5.591997672459713, 5.59111...","[51.99416434974128, 52.00139228996948, 52.0073..."
4,Groningen,"[6.286980744809261, 6.273680380337391, 6.25404...","[53.34138175175612, 53.345271499820676, 53.348..."
5,Limburg,"[5.919469547636648, 5.899566718712435, 5.88663...","[51.71767149749075, 51.720189936851206, 51.726..."
6,Noord-Brabant,"[5.206494174004241, 5.216347456767744, 5.22503...","[51.74147718300763, 51.74345117135748, 51.7429..."
7,Noord-Holland,"[4.583320874314651, 4.596585061787816, 4.61146...","[52.53389206786423, 52.58400753386461, 52.6555..."
8,Overijssel,"[6.109580017263029, 6.102294636279321, 6.10335...","[52.440530726348904, 52.44577189750357, 52.456..."
9,Utrecht,"[4.892178256568383, 4.855581829109592, 4.84392...","[52.16179791879903, 52.178917173752055, 52.180..."


### 3d: Samenvoegen

In [ ]:
huisjes_voorzieningen_woonplaatsen = pd.merge(huisjes, voorzieningen_en_woonplaatsen, on = 'Woonplaats', how = 'inner')
huisjes_voorzieningen_woonplaatsen

,Natuurhuisje-ID,Woonplaats,Regio,Land,Aantal personen,Slaapkamers,Type,Huursituatie,Ligging,Check-in,Check-out,Departement,Woonplaatscode,AfstandTotGroteSupermarkt_20,AfstandTotTreinstationsTotaal_101,AfstandTotZwembad_104,AfstandTotBioscoop_115,AfstandTotSauna_119,AfstandTotAttractie_121
0,71499,Putten,Gelderland,Nederland,6,2,Glamping,Vrijstaand,Op een landgoed,16:00 - 21:00,08:00 - 11:00,NaN,GM0273,1.5,2.9,3.0,12.3,6.2,5.1
1,67787,Putten,Gelderland,Nederland,4,2,Chalet,Vrijstaand,Kleinschalig vakantiepark,15:00 - 22:00,07:00 - 10:30,NaN,GM0273,1.5,2.9,3.0,12.3,6.2,5.1
2,64897,Putten,Gelderland,Nederland,4,2,Bungalow,Vrijstaand,Kleinschalig vakantiepark,15:00 - 22:00,07:00 - 10:00,NaN,GM0273,1.5,2.9,3.0,12.3,6.2,5.1
3,65821,Putten,Gelderland,Nederland,4,2,Vakantiehuis,Vrijstaand,Kleinschalig vakantiepark,16:00 - 18:00,07:00 - 10:00,NaN,GM0273,1.5,2.9,3.0,12.3,6.2,5.1
4,64745,Sevenum,Limburg,Nederland,2,1,Boerderij,Vrijstaand,Alleenstaand,15:00 - 21:00,09:00 - 11:00,NaN,GM0964,NaN,NaN,NaN,NaN,NaN,NaN
5,62656,Rucphen,Noord-Brabant,Nederland,6,3,Vakantiehuis,Vrijstaand,Op een erf,16:00 - 21:30,09:00 - 10:00,NaN,GM0840,0.8,6.5,2.8,6.2,12.2,5.4
6,64548,Ermelo,Gelderland,Nederland,2,1,Vakantiehuis,Vrijstaand,Op een erf,15:00 - 20:00,09:00 - 11:00,NaN,GM0233,1.1,2.0,2.2,6.7,9.9,4.0
7,67417,Haaksbergen,Overijssel,Nederland,2,1,Vakantiehuis,Vrijstaand,Op een erf,15:00 - 20:00,07:00 - 10:00,NaN,GM0158,1.3,12.6,3.0,13.7,17.7,11.2
8,63898,Epe,Gelderland,Nederland,4,1,Vakantiehuis,Vrijstaand,Op een landgoed,16:00 - 21:00,11:00 - 12:00,NaN,GM0232,1.2,10.4,5.4,13.6,5.1,4.5
9,55143,Epe,Gelderland,Nederland,4,2,Vakantiehuis,Geschakeld,Op een erf,15:00 - 18:00,07:00 - 10:00,NaN,GM0232,1.2,10.4,5.4,13.6,5.1,4.5


In [ ]:
huisjes_voorzieningen_woonplaatsen = huisjes_voorzieningen_woonplaatsen.groupby('Regio')['Natuurhuisje-ID', 'AfstandTotGroteSupermarkt_20'].agg(['count', 'mean']).reset_index()
huisjes_voorzieningen_woonplaatsen = huisjes_voorzieningen_woonplaatsen.iloc[:, [0, 1, 4]]
huisjes_voorzieningen_woonplaatsen.columns = ['Provincie', 'aantal_huisjes', 'gemiddelde_afstand_tot_grote_supermarkt']
huisjes_voorzieningen_woonplaatsen

,Provincie,aantal_huisjes,gemiddelde_afstand_tot_grote_supermarkt
0,Drenthe,1,1.000000
1,Flevoland,1,1.100000
2,Friesland,2,0.900000
3,Gelderland,18,1.266667
4,Groningen,2,NaN
5,Limburg,3,1.050000
6,Noord-Brabant,2,0.900000
7,Noord-Holland,2,0.900000
8,Overijssel,8,1.375000
9,Utrecht,1,1.000000


In [ ]:
huisjes_voorzieningen_woonplaatsen_provincies = pd.merge(provincies, huisjes_voorzieningen_woonplaatsen, how = 'left', left_on = 'name', right_on = 'Provincie')
huisjes_voorzieningen_woonplaatsen_provincies

,name,x,y,Provincie,aantal_huisjes,gemiddelde_afstand_tot_grote_supermarkt
0,Drenthe,"[6.413279838107642, 6.36252137268322, 6.367810...","[52.98552292584422, 53.033969027608755, 53.067...",Drenthe,1.0,1.000000
1,Flevoland,"[5.361146853111475, 5.377259769555099, 5.63135...","[52.67572642054404, 52.76480520148312, 52.8030...",Flevoland,1.0,1.100000
2,Friesland,"[5.087068926159327, 5.101778301669335, 5.16177...","[53.3230724825529, 53.36803427684854, 53.38613...",Friesland,2.0,0.900000
3,Gelderland,"[5.606017014104991, 5.591997672459713, 5.59111...","[51.99416434974128, 52.00139228996948, 52.0073...",Gelderland,18.0,1.266667
4,Groningen,"[6.286980744809261, 6.273680380337391, 6.25404...","[53.34138175175612, 53.345271499820676, 53.348...",Groningen,2.0,NaN
5,Limburg,"[5.919469547636648, 5.899566718712435, 5.88663...","[51.71767149749075, 51.720189936851206, 51.726...",Limburg,3.0,1.050000
6,Noord-Brabant,"[5.206494174004241, 5.216347456767744, 5.22503...","[51.74147718300763, 51.74345117135748, 51.7429...",Noord-Brabant,2.0,0.900000
7,Noord-Holland,"[4.583320874314651, 4.596585061787816, 4.61146...","[52.53389206786423, 52.58400753386461, 52.6555...",Noord-Holland,2.0,0.900000
8,Overijssel,"[6.109580017263029, 6.102294636279321, 6.10335...","[52.440530726348904, 52.44577189750357, 52.456...",Overijssel,8.0,1.375000
9,Utrecht,"[4.892178256568383, 4.855581829109592, 4.84392...","[52.16179791879903, 52.178917173752055, 52.180...",Utrecht,1.0,1.000000


In [ ]:
huisjes_voorzieningen_woonplaatsen_provincies['aantal_huisjes'] = huisjes_voorzieningen_woonplaatsen_provincies['aantal_huisjes'].fillna(0)
huisjes_voorzieningen_woonplaatsen_provincies
#TODO de kaart maakt alles wit als er minstens één 0 in 'aantal_huisjes' zit (als gevolg van de fillna). Oplossen.

,name,x,y,Provincie,aantal_huisjes,gemiddelde_afstand_tot_grote_supermarkt
0,Drenthe,"[6.413279838107642, 6.36252137268322, 6.367810...","[52.98552292584422, 53.033969027608755, 53.067...",Drenthe,1.0,1.000000
1,Flevoland,"[5.361146853111475, 5.377259769555099, 5.63135...","[52.67572642054404, 52.76480520148312, 52.8030...",Flevoland,1.0,1.100000
2,Friesland,"[5.087068926159327, 5.101778301669335, 5.16177...","[53.3230724825529, 53.36803427684854, 53.38613...",Friesland,2.0,0.900000
3,Gelderland,"[5.606017014104991, 5.591997672459713, 5.59111...","[51.99416434974128, 52.00139228996948, 52.0073...",Gelderland,18.0,1.266667
4,Groningen,"[6.286980744809261, 6.273680380337391, 6.25404...","[53.34138175175612, 53.345271499820676, 53.348...",Groningen,2.0,NaN
5,Limburg,"[5.919469547636648, 5.899566718712435, 5.88663...","[51.71767149749075, 51.720189936851206, 51.726...",Limburg,3.0,1.050000
6,Noord-Brabant,"[5.206494174004241, 5.216347456767744, 5.22503...","[51.74147718300763, 51.74345117135748, 51.7429...",Noord-Brabant,2.0,0.900000
7,Noord-Holland,"[4.583320874314651, 4.596585061787816, 4.61146...","[52.53389206786423, 52.58400753386461, 52.6555...",Noord-Holland,2.0,0.900000
8,Overijssel,"[6.109580017263029, 6.102294636279321, 6.10335...","[52.440530726348904, 52.44577189750357, 52.456...",Overijssel,8.0,1.375000
9,Utrecht,"[4.892178256568383, 4.855581829109592, 4.84392...","[52.16179791879903, 52.178917173752055, 52.180...",Utrecht,1.0,1.000000


## Fase 4: Evaluation & Deployment

In [ ]:
TOOLS = 'pan, wheel_zoom, reset, hover, save'

In [ ]:
p = figure(
    title = 'Nederlandse Provincies', tools = TOOLS,
    x_axis_location = None,
    y_axis_location = None,
    tooltips = [
        ('Name', '@name'),
        ('Aantal huisjes', '@aantal_huisjes'),
        ("Long", "$x"),
        ("Lat", '$y')
    ]
)

p

Figure(id='4432', ...)

In [ ]:
p.grid.grid_line_color = None
p.hover.point_policy = 'follow_mouse'

p.patches('x',
          'y',
          source = ColumnDataSource(huisjes_voorzieningen_woonplaatsen_provincies),
          fill_color = {'field' : 'aantal_huisjes', 'transform' : LogColorMapper(palette = tuple(reversed(correct_palette[9])))},
          fill_alpha = 0.6,
          line_color = 'black',
          line_width = 0.5)

show(p)